In [1]:
import json
import requests as rq
import pandas as pd
import numpy as np
import time
import itertools
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
G=nx.Graph()
import pymongo
from pymongo import MongoClient
import community

list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheetname = 'table_output',
                      header = 0,
                      index_col = 0,
                      parse_cols = "A",
                      convert_float = True)
len(list_summoners)

summ_file = pd.read_excel('LVP_Working_File_Simple.xlsx',
                      sheetname = 'Sheet1',
                      header = 0,
                      parse_cols = "A:E",
                      convert_float = True, index=False)

C:\ProgramData\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
C:\ProgramData\Anaconda\envs\py36\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\ProgramData\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead


In [2]:
def most_important(H):
    """ returns a copy of G with
     the most important nodes
     according to the pagerank """ 
    ranking = nx.betweenness_centrality(H).items()
    r = [x[1] for x in ranking]
    m = sum(r)/len(r) # mean centrality
    t = m*3 # threshold, we keep only the nodes with 3 times the mean
    Gt = H.copy()
    for k, v in ranking:
        if v < t:
            Gt.remove_node(k)
    return Gt, m

def most_important_w(H):
    """ returns a copy of G with
     the most important nodes
     according to the pagerank """ 
    ranking = nx.betweenness_centrality(H, weight= 'matches_together').items()
    r = [x[1] for x in ranking]
    m = sum(r)/len(r) # mean centrality
    t = m*3 # threshold, we keep only the nodes with 3 times the mean
    Gt = H.copy()
    for k, v in ranking:
        if v < t:
            Gt.remove_node(k)
    return Gt, m

In [3]:
list_records = []

G = nx.read_graphml("networks/playerNetwork_"+ str(20501992) +".graphml")

# Características globales e id
ego_id = G.graph['ego_id']
num_nodes = len(list(G.nodes()))

# Elimino el ego
G.remove_node(G.graph['ego_id'])

# Partition Louvain
partition = community.best_partition(G)
num_communities = len(set(partition.values()))
modularity = community.modularity(partition, G)    

# Measures
num_components = nx.number_connected_components(G)
density = nx.density(G)
node_coverage = len(list(G.nodes()))/(G.graph['total_matches']*4)

# Giant component
Gcc = sorted(nx.connected_component_subgraphs(G), key=len, reverse=True)
G0 = Gcc[0]
giant_component_coverage = len(list(G0.nodes()))/len(list(G.nodes()))

# Avg Degree
degrees = G.degree()
sum_of_edges = 0
for a, b in list(degrees):
    sum_of_edges = sum_of_edges + b
avg_degree = sum_of_edges/len(list(G.nodes()))

# Betweenness Centrality (most important nodes)
Gt, m = most_important(G)
num_important_nodes = len(list(Gt))
perc_important_nodes = len(list(Gt))/len(list(G.nodes()))
mean_centrality = m


list_records.append((ego_id, num_nodes, num_communities, modularity, num_components, density, 
                     node_coverage, giant_component_coverage, avg_degree, perc_important_nodes, mean_centrality))



In [4]:
list_records

[('20501992',
  22,
  3,
  0.5382653061224489,
  2,
  0.19047619047619047,
  0.75,
  0.8095238095238095,
  3.8095238095238093,
  0.09523809523809523,
  0.03609022556390978)]

# Genero el random

In [5]:
G.degree()

DegreeView({'61521191': 3, '53037561': 7, '31113043': 10, '35773403': 3, '52662940': 3, '37945613': 3, '52493865': 3, '24658882': 3, '63986793': 3, '22003918': 3, '182419': 9, '40638373': 3, '25914000': 3, '24818653': 3, '21137760': 3, '50766893': 3, '37742947': 3, '44929492': 3, '37158550': 3, '33011554': 3, '52395740': 3})

In [ ]:
list_records = []
for i in range(len(list_summoners)):
    G = nx.read_graphml("networks/playerNetwork_"+ str(list_summoners.index[i]) +".graphml")
    
    # Características globales e id
    ego_id = G.graph['ego_id']
    num_nodes = len(list(G.nodes()))
    
    # Elimino el ego
    G.remove_node(G.graph['ego_id'])
    
    # Partition Louvain
    partition = community.best_partition(G)
    num_communities = len(set(partition.values()))
    modularity = community.modularity(partition, G)    
    
    # Measures
    num_components = nx.number_connected_components(G)
    density = nx.density(G)
    node_coverage = len(list(G.nodes()))/(G.graph['total_matches']*4)
    
    # Giant component
    Gcc = sorted(nx.connected_component_subgraphs(G), key=len, reverse=True)
    G0 = Gcc[0]
    giant_component_coverage = len(list(G0.nodes()))/len(list(G.nodes()))
    
    # Avg Degree
    degrees = G.degree()
    sum_of_edges = 0
    for a, b in list(degrees):
        sum_of_edges = sum_of_edges + b
    avg_degree = sum_of_edges/len(list(G.nodes()))
    
    # Betweenness Centrality (most important nodes)
    Gt, m = most_important(G)
    num_important_nodes = len(list(Gt))
    perc_important_nodes = len(list(Gt))/len(list(G.nodes()))
    mean_centrality = m
    
    
    list_records.append((ego_id, num_nodes, num_communities, modularity, num_components, density, 
                         node_coverage, giant_component_coverage, avg_degree, perc_important_nodes, mean_centrality))
    
    